In [17]:
def create_features(years, lags):

    import pandas as pd 

    ID = pd.read_pickle(f"../epex/ID.pkl")

    ID = ID.astype('float')

    ID = ID.loc[ID.index.year.isin(years)]

    DA = pd.read_pickle(f"../epex/DA.pkl")

    DA = DA.astype('float')

    DA = DA.loc[DA.index.year.isin(years)]

    features = pd.merge(ID, DA, left_index=True, right_index=True, how='outer')

    for l in lags:
        features[f'ID3 ({l})'] = features['ID3'].shift(l)

    features = features.dropna()

    return ID, DA, features